In [7]:
from Model_Training import Model_Training
from Cleaner import Cleaner
import plotly.express as px
import warnings
import pandas as pd
from EmpiricDistribution import EmpiricDistribution
from datetime import datetime, time
import pickle # Para guardar los modelos entrenados
from datetime import time 
import matplotlib.pyplot as plt
import numpy as np
import random
from sympy import Symbol
from datetime import time
import random

model = Model_Training()

### Buses

In [2]:
#Uncomment this line if you dont have the time_models.pkl

#model.generate_models()

c:\Users\crdie\Documents\Tec\5Semestre\Optimizacion\DiscreteSimulation-TransporTec\Cleaner.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['trayecto'] = df_final['inicio'] + '-'+ df_final['final']


Iniciando modelado de la distribución de Tiempos de Viaje...
Error: No valid symbolic solution found (likely m=0) for interval 10. Using x_max as inverse.
Error: No valid symbolic solution found (likely m=0) for interval 13. Using x_max as inverse.
Error: No valid symbolic solution found (likely m=0) for interval 14. Using x_max as inverse.
  -> Modelo de Tiempo A2-GLAXO_0700-0930 entrenado.
Error: No valid symbolic solution found (likely m=0) for interval 13. Using x_max as inverse.
Error: No valid symbolic solution found (likely m=0) for interval 14. Using x_max as inverse.
  -> Modelo de Tiempo A2-GLAXO_1330-1800 entrenado.
  -> Modelo de Tiempo A2-GLAXO_0600-0700 entrenado.
Error: No valid symbolic solution found (likely m=0) for interval 9. Using x_max as inverse.
Error: No valid symbolic solution found (likely m=0) for interval 10. Using x_max as inverse.
Error: No valid symbolic solution found (likely m=0) for interval 11. Using x_max as inverse.
Error: No valid symbolic solutio

In [3]:
#Uncomment this line if you dont have the simulation_results_times.csv

#model.generate_simulation()

Modelos de tiempo cargados correctamente.

Iniciando simulación de Monte Carlo (Solo Tiempos) con 1000 iteraciones...
  -> Segmento A2-GLAXO_0700-0930 simulado con 1000 muestras.
  -> Segmento A2-GLAXO_1330-1800 simulado con 1000 muestras.
  -> Segmento A2-GLAXO_0600-0700 simulado con 1000 muestras.
  -> Segmento A2-GLAXO_0930-1330 simulado con 1000 muestras.
  -> Sustitución TIEMPO en A2-GLAXO_1830-2215: Usando proxy A2-GLAXO_0600-0700.
  -> Segmento GLAXO-DISNEY_0700-0930 simulado con 1000 muestras.
  -> Segmento GLAXO-DISNEY_1330-1800 simulado con 1000 muestras.
  -> Segmento GLAXO-DISNEY_0600-0700 simulado con 1000 muestras.
  -> Segmento GLAXO-DISNEY_0930-1330 simulado con 1000 muestras.
  -> Sustitución TIEMPO en GLAXO-DISNEY_1830-2215: Usando proxy A2-GLAXO_0600-0700.
  -> Segmento DISNEY-A2_0700-0930 simulado con 1000 muestras.
  -> Segmento DISNEY-A2_1330-1800 simulado con 1000 muestras.
  -> Segmento DISNEY-A2_0600-0700 simulado con 1000 muestras.
  -> Segmento DISNEY-A2_0930

In [5]:
#model.show_results()

### PAX

In [8]:
cleaner = Cleaner()

cleaner.set_path_info({'7': '07_v2.xlsx', '50':'50.xlsx'})

df = cleaner.process_data(export_csv=False)

df['hora_salida'] = pd.to_datetime(df['hora_salida'], format='%H:%M').dt.time
df = df.dropna(subset=['Usuarios'])

c:\Users\crdie\Documents\Tec\5Semestre\Optimizacion\DiscreteSimulation-TransporTec\Cleaner.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['trayecto'] = df_final['inicio'] + '-'+ df_final['final']


In [21]:
#Calculating lambdas per interval and place
TRAYECTOS = ['A2-GLAXO', 'GLAXO-DISNEY', 'DISNEY-A2']
TIME_INTERVALS = [
    ('07:00', '09:30'), 
    ('13:30', '18:00'), 
    ('06:00', '07:00'), 
    ('09:30', '13:30'), 
    ('18:30', '22:15') 
]
N_BUSES = 2

lambdas_saved = {}

REF_START, REF_END = '13:30', '18:00'
TARGET_START, TARGET_END = '18:30', '22:15'

for start_time, end_time in TIME_INTERVALS:
    sub_df = df.loc[(df['hora_salida'].astype(str) >= start_time) & (df['hora_salida'].astype(str) < end_time)]
    avg_cycle_time = sub_df.groupby('trayecto')['minutos_viaje'].mean().sum()

    for t in TRAYECTOS:
        key = f"{t}_{start_time.replace(':', '')}-{end_time.replace(':', '')}"
        lambda_ = 0
        if(start_time == TARGET_START):
            #special case, 20% less than the ('13:30', '18:00')
            lambda_ = lambdas_saved[f"{t}_{REF_START.replace(':', '')}-{REF_END.replace(':', '')}"]*.8
        else:
            avg_pax_in_bus = sub_df.loc[sub_df['trayecto'] == t]['Usuarios'].mean()
            print(avg_pax_in_bus, f'for key: {key}')
            # Lambda = avg_pax / (cycle_time/n_buses)
            lambda_ = avg_pax_in_bus / (avg_cycle_time/N_BUSES)

        lambdas_saved[key] = lambda_

2.8508771929824563 for key: A2-GLAXO_0700-0930
0.8526785714285714 for key: GLAXO-DISNEY_0700-0930
8.018604651162791 for key: DISNEY-A2_0700-0930
8.584905660377359 for key: A2-GLAXO_1330-1800
2.010152284263959 for key: GLAXO-DISNEY_1330-1800
4.972850678733032 for key: DISNEY-A2_1330-1800
3.4468085106382977 for key: A2-GLAXO_0600-0700
0.46296296296296297 for key: GLAXO-DISNEY_0600-0700
2.3333333333333335 for key: DISNEY-A2_0600-0700
4.6770428015564205 for key: A2-GLAXO_0930-1330
1.3793103448275863 for key: GLAXO-DISNEY_0930-1330
6.504424778761062 for key: DISNEY-A2_0930-1330


In [29]:
#Asumiendo que es un Proceso de Poisson, entonces los tiempos de llegada se comportan como una exponencial

n_samples = 10000

data_simulated = {}

for start_time, end_time in TIME_INTERVALS:
    for t in TRAYECTOS:
        for _ in range(n_samples):
            key = f"{t}_{start_time.replace(':', '')}-{end_time.replace(':', '')}"
            lambda_act = lambdas_saved[key]
            t_arrival = random.expovariate(lambda_act)
            if key not in data_simulated:
                data_simulated[key] = []
            data_simulated[key].append(t_arrival)


df_pax_simulated = pd.DataFrame(data_simulated)

df_pax_simulated.to_csv('pax_time_arrivals.csv', index=False)

### DISCRETE SIMULATION